## Getting started
Set working directory to top level of repo to ensure links work correctly:

In [1]:
cd ..

/home/jovyan/Robbi/dea-intertidal


In [2]:
pip install odc-geo --quiet

You should consider upgrading via the '/env/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install git+https://github.com/digitalearthafrica/deafrica-coastlines.git --quiet

You should consider upgrading via the '/env/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


### Load packages

In [4]:
%load_ext autoreload
%autoreload 2

import datacube
import xarray as xr
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from datacube.utils.cog import write_cog
from datacube.utils.geometry import Geometry
import odc.geo.xr
from odc.algo import xr_geomedian

from dea_tools.dask import create_local_dask_cluster, create_dask_gateway_cluster
from intertidal.extents import load_data, pixel_tide_sort, item, nidem, parallel_apply, pixel_tides

# Create local dask cluster to improve data load time
client = create_local_dask_cluster(return_client=True)

Client Scheduler: tcp://127.0.0.1:45361 Dashboard: /user/robbi.bishoptaylor@ga.gov.au/proxy/8787/status,Cluster Workers: 1 Cores: 7 Memory: 63.57 GB


## Setup

In [5]:
dc = datacube.Datacube(app='Intertidal_elevation')

In [ ]:
# aoi = 'McCarthur River'
# aoi = 'Greenbank'
# aoi = 'Seven Emu'
aoi = 'Wollogorang'

# Import regions of interest
aoi_gdf = gpd.read_file('data/raw/Phase_1_NT_Gov.geojson').set_index('Name').to_crs('EPSG:3577')
aoi_gdf.head()

# Select AOI
geom = Geometry(geom=aoi_gdf.loc[aoi].geometry, crs='EPSG:3577')
geom

In [ ]:
from odc.ui import select_on_a_map
from ipyleaflet import basemaps, basemap_to_tiles

# Plot interactive map to select area
basemap = basemap_to_tiles(basemaps.Esri.WorldImagery)
geom = select_on_a_map(height='600px',
                             layers=(basemap,),
                             center=(-26, 135), 
                             zoom=4)

Map(center=[-26, 135], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

## Load data

In [15]:
ds = load_data(dc=dc, 
               geom=geom, 
               time_range=('2019', '2021'), 
               resolution=10, 
               s2_prod="s2_nbart_norm",
               ls_prod="ls_nbart_norm")
ds

<xarray.Dataset>
Dimensions:      (time: 438, y: 2726, x: 4813)
Coordinates:
  * time         (time) datetime64[ns] 2019-01-02T00:59:08.684254 ... 2021-12...
  * y            (y) float64 8.261e+06 8.261e+06 ... 8.234e+06 8.234e+06
  * x            (x) float64 6.743e+05 6.743e+05 ... 7.224e+05 7.224e+05
    spatial_ref  int32 32753
Data variables:
    blue         (time, y, x) float32 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    green        (time, y, x) float32 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    red          (time, y, x) float32 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    nir          (time, y, x) float32 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    swir1        (time, y, x) float32 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    swir2        (time, y, x) float32 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
Attributes:
    crs:           epsg:32753
    grid_mapping:  spatial_ref

## Pixel-based tides

In [16]:
# Model tides into every pixel in the dataset, and set tide pixels to nodata
# TODO: Work out why this works if `_reproject` is defined here, but fails if
# `_reproject` is defined in script
def _reproject(x, to_geobox=ds.odc.geobox):
    return x.odc.reproject(to_geobox, resampling="bilinear")

ds["tide_m"], _ = pixel_tides(ds, resample_func=_reproject, directory='~/tide_models_clipped')

# Set tide pixels to nodata if input array has nodata
# ds["tide_m"] = ds["tide_m"].where(~ds.to_array().isel(variable=0).isnull())
# ds = ds.chunk({'y': 2048, 'x': 2048})


Rescaling and flattening tide modelling array
Modelling tides
Unstacking tide modelling array
Reprojecting tides into original array


100%|██████████| 438/438 [01:01<00:00,  7.15it/s]


## HLTC

In [17]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/robbi.bishoptaylor@ga.gov.au/proxy/8787/status,
Dashboard: /user/robbi.bishoptaylor@ga.gov.au/proxy/8787/status,Workers: 1
Total threads: 62,Total memory: 477.21 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40875,Workers: 1
Dashboard: /user/robbi.bishoptaylor@ga.gov.au/proxy/8787/status,Total threads: 62
Started: 17 minutes ago,Total memory: 477.21 GiB
Comm: tcp://127.0.0.1:38703,Total threads: 62
Dashboard: /user/robbi.bishoptaylor@ga.gov.au/proxy/36037/status,Memory: 477.21 GiB
Nanny: tcp://127.0.0.1:44459,


In [18]:
ds.persist()

<xarray.Dataset>
Dimensions:      (time: 438, y: 2726, x: 4813)
Coordinates:
  * time         (time) datetime64[ns] 2019-01-02T00:59:08.684254 ... 2021-12...
  * y            (y) float64 8.261e+06 8.261e+06 ... 8.234e+06 8.234e+06
  * x            (x) float64 6.743e+05 6.743e+05 ... 7.224e+05 7.224e+05
    spatial_ref  int32 32753
Data variables:
    blue         (time, y, x) float32 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    green        (time, y, x) float32 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    red          (time, y, x) float32 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    nir          (time, y, x) float32 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    swir1        (time, y, x) float32 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    swir2        (time, y, x) float32 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    tide_m       (time, y, x) float32 0.1413 0.1413 0.1413 ... 0.07241 0.07241
Attributes:
    crs:           epsg:32753
    grid_mapping:  spatial_ref

In [19]:
# Calculate max, min and full range of tide
tide_max = ds.tide_m.max(dim='time')
tide_min = ds.tide_m.min(dim='time')
tide_range = tide_max - tide_min

In [20]:
# Calculate a threshold for low tide composite
min_thresh = tide_min + (tide_range * 0.2)
ds_min = ds.where(ds.tide_m <= min_thresh)
ds_min = ds_min.sel(time = ds_min.tide_m.isnull().mean(dim=['x', 'y']) < 1).drop('tide_m')
ds_min

<xarray.Dataset>
Dimensions:      (time: 46, y: 2726, x: 4813)
Coordinates:
  * time         (time) datetime64[ns] 2019-01-10T00:54:13.477130 ... 2021-12...
  * y            (y) float64 8.261e+06 8.261e+06 ... 8.234e+06 8.234e+06
  * x            (x) float64 6.743e+05 6.743e+05 ... 7.224e+05 7.224e+05
    spatial_ref  int32 32753
Data variables:
    blue         (time, y, x) float32 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    green        (time, y, x) float32 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    red          (time, y, x) float32 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    nir          (time, y, x) float32 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    swir1        (time, y, x) float32 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    swir2        (time, y, x) float32 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
Attributes:
    crs:           epsg:32753
    grid_mapping:  spatial_ref

In [21]:
# Compute geomedian
ds_min_median = xr_geomedian(ds=ds_min)
ds_min_median.load()
ds_min_median.to_array().odc.write_cog('hltc_20_s2ls.tif', overwrite=True)

PosixPath('hltc_20_s2ls.tif')

In [52]:
# Calculate a threshold for low tide composite
min_thresh = tide_min + (tide_range * 0.8)
ds_max = ds.where(ds.tide_m >= min_thresh)
ds_max = ds_max.sel(time = ds_max.tide_m.isnull().mean(dim=['x', 'y']) < 1).drop('tide_m')

# Compute geomedian
ds_max_median = xr_geomedian(ds=ds_max)
ds_max_median.load()

<xarray.Dataset>
Dimensions:  (y: 1382, x: 2080)
Coordinates:
  * y        (y) float64 8.304e+06 8.304e+06 8.304e+06 ... 8.29e+06 8.29e+06
  * x        (x) float64 6.166e+05 6.166e+05 6.166e+05 ... 6.373e+05 6.374e+05
Data variables:
    blue     (y, x) float32 0.0588 0.0589 0.05874 ... 0.04827 0.04911 0.04903
    green    (y, x) float32 0.07444 0.07457 0.07454 ... 0.04495 0.0458 0.0454
    red      (y, x) float32 0.038 0.03777 0.0376 ... 0.01412 0.01428 0.0146
    nir      (y, x) float32 0.009966 0.01004 0.00992 ... 0.008547 0.007916
    swir1    (y, x) float32 0.006353 0.006334 0.006326 ... 0.004625 0.004582
    swir2    (y, x) float32 0.006353 0.006334 0.006326 ... 0.004625 0.004582

In [54]:

ds_max_median.to_array().odc.write_cog('hltc_80.tif', overwrite=True)

PosixPath('hltc_80.tif')

In [ ]:
# ds_min_median = ds_min[['swir1', 'nir', 'green']].median(dim='time')

In [ ]:
ds_min_median.odc.to_rgba(vmin=0, vmax=0.3).plot.imshow(size=10)

In [ ]:
ds_min_median

In [ ]:
ds_min_median.to_array().odc.write_cog('hltc_10.tif')

In [ ]:
ds_min_median.odc.to_rgba(bands=['swir1', 'nir', 'green'], vmin=0, vmax=0.3).plot.imshow(size=10)

In [ ]:
# Mask to only good pixels, stack to 1D, then drop obs without data
ds_flat = ds.where(good_mask).stack(z=("x", "y")).dropna(dim='z', how='all')


In [ ]:
# Calculate tide min, max, range
tide_max = ds_flat.tide_m.max(dim='time')
tide_min = ds_flat.tide_m.min(dim='time')
tide_range = tide_max - tide_min

# Choose number of rolling window intervals and window radius
window_interval_n = 100  # number of window intervals to iterate over
window_tide_prop = 0.15  # proportion of the tide range to use as window radius
window_tide = tide_range * window_tide_prop  # window radius in tide/metre units
window_interval_tide = tide_range / window_interval_n  # interval size in time/metre units
window_interval_buffer = int((window_interval_n * window_tide_prop) / 2.0)


In [ ]:
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
from odc.algo import xr_quantile

def rolling_tide_window(i, ds=ds_flat, interval_tide=window_interval_tide, interval_window=window_tide):    
   
    # Set min and max thresholds to filter dataset
    thresh_centre = tide_min + (i * interval_tide)
    thresh_min = thresh_centre - interval_window
    thresh_max = thresh_centre + interval_window
    
    # Filter dataset
    masked_ds = ds.where((ds.tide_m >= thresh_min) & (ds.tide_m <= thresh_max))
    
    # Apply median or quantile
    ds_median = masked_ds.median(dim='time')
    # ds_median = masked_ds.quantile(q=[0.1, 0.5, 0.9], dim='tide_n')
    # ds_median = xr_quantile(src=masked_ds, quantiles=[0.1, 0.5, 0.9], nodata=np.nan)
    
    # Add standard deviation
    ds_median['ndwi_std'] = masked_ds.ndwi.std(dim='time')
#     ds_median['ndwi_count'] = (~masked_ds.ndwi.isnull()).sum(dim='time')

    return ds_median

with ProcessPoolExecutor() as executor:
    
    # Apply func in parallel
    rolling_intervals = range(-window_interval_buffer, window_interval_n + window_interval_buffer)
    out_list = tqdm(executor.map(rolling_tide_window, rolling_intervals), total=len(list(rolling_intervals)))

    # Combine to match the original dataset
    interval_ds = xr.concat(out_list, dim='interval').sortby(['interval', 'x', 'y'])

In [ ]:
# Output name
fname = "testing4"
ndwi_thresh = 0.1
# fname = aoi.lower().replace(' ', '')

# Calculate confidence (mean of NDWI standard deviation)
confidence = interval_ds.ndwi_std.mean(dim="interval").unstack("z").reindex_like(ds).T

# Add quantile dim if it does not exist
interval_ds = (
    interval_ds.expand_dims(quantile=[0.5])
    if "quantile" not in interval_ds
    else interval_ds
)

# Export DEM for each quantile
for q in interval_ds["quantile"].data:

    # Extract relevant quantile data
    print(f"Processing quantile {q}")
    quantile_ds = interval_ds.sel(quantile=q)

    # Identify the max tide per pixel where NDWI == land
    tide_dry = quantile_ds.tide_m.where(
        quantile_ds.ndwi <= ndwi_thresh
    )  # .dropna(dim='z', how='all')
    tide_thresh = tide_dry.max(dim="interval")
    #     tide_argmax = tide_dry.argmax(dim='interval')
    #     tide_thresh = tide_dry.isel(interval=(tide_argmax))
    #     tide_thresh = tide_dry.isel(interval=(tide_argmax + 1).clip(0, tide_argmax.max().item()))
    tide_max = quantile_ds.tide_m.max(dim="interval")

    # Remove any pixel where tides max out (i.e. always land), and unstack back
    # to 3D array
    always_dry = tide_thresh >= tide_max
    dem = tide_thresh.where(~always_dry)
    dem = dem.unstack("z").reindex_like(ds).T

    # Export DEM file
    suffix = {0.1: "_low", 0.5: "", 0.9: "_high"}[q]
    dem.odc.write_cog(
        fname=f"data/interim/pixel_{fname}_dem{suffix}.tif", overwrite=True
    )

# Export NDWI standard deviation/confidence file
confidence.odc.write_cog(
    fname=f"data/interim/pixel_{fname}_confidence.tif", overwrite=True
)